In [1]:
# ENSEMBLE MODELS

In [2]:


import sys
import os
import wandb
import pandas as pd
sys.path.append('/u/home/koksal/organ-mesh-registration-and-property-prediction/')

import torch
from src.models.fsgn_model import MeshSeg
from src.data.organs_dataset import OrganMeshDataset
from torch_geometric.data import DataLoader
from tqdm import tqdm
from time import sleep
import mlflow
from src.models.baseline_model import GNN
import argparse
from sklearn.metrics import r2_score

#config = Args()
from src.models.train import build_dataset, build_network
from src.models.train import accuracy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
class Dict2Class(object):
    def __init__(self, my_dict):
        for key in my_dict:
            setattr(self, key, my_dict[key])

In [4]:
device='cuda:1'

In [5]:
def return_data_model(experiment_dir, experiment_name, organ='spleen', return_dataset=True):

    checkpoint = torch.load(os.path.join(experiment_dir, experiment_name))
    config = checkpoint['config']
    config = Dict2Class(config)

    model = build_network(config).to(device)
    model.load_state_dict(checkpoint['model'])

    dataset_config_dict = {
        "decimation_path":"/data0/practical-wise2223/organ_mesh/organ_decimations_ply/",
        "registeration_path" : "/data0/practical-wise2223/organ_mesh/gendered_organ_registrations_ply/",
        "split_path" : '/data0/practical-wise2223/organ_mesh/data/',
        "root" : '/data0/practical-wise2223/organ_mesh/organ_meshes',
        "basic_feat_path" : '/data0/practical-wise2223/organ_mesh/basic_features.csv',
        "bridge_path" : '/data0/practical-wise2223/organ_mesh/Bridge_eids_60520_87802.csv',
        "num_train_samples": 3000,
        "num_test_samples":300,
        'batch_size': 2,
        'task': 'bmi_prediction',
        'use_registered_data': False,
        "organ": organ
    }
    data_config = Dict2Class(dataset_config_dict)
    #Data Loader
    train_loader, test_loader = build_dataset(data_config,return_dataset)
    #Network

    return model, test_loader


In [6]:
dataset_config_dict = {
    "decimation_path":"/data0/practical-wise2223/organ_mesh/organ_decimations_ply/",
    "registeration_path" : "/data0/practical-wise2223/organ_mesh/gendered_organ_registrations_ply/",
    "split_path" : '/data0/practical-wise2223/organ_mesh/data/',
    "root" : '/data0/practical-wise2223/organ_mesh/organ_meshes',
    "basic_feat_path" : '/data0/practical-wise2223/organ_mesh/basic_features.csv',
    "bridge_path" : '/data0/practical-wise2223/organ_mesh/Bridge_eids_60520_87802.csv',
    "num_train_samples": 3000,
    "num_test_samples":300,
    'batch_size': 3000,
    'task': 'bmi_prediction',
    'use_registered_data': False,
    "organ": 'liver',
    "use_scaled_data": True,
}
data_config = Dict2Class(dataset_config_dict)
#Data Loader
train_loader, test_loader = build_dataset(data_config,return_dataset=True)

Liver  train Dataset is created
Liver  val Dataset is created
Liver  test Dataset is created


/u/home/koksal/.conda/envs/caghan4/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [7]:
# LIVER MODEL
liver_exp_dir = '/u/home/koksal/organ-mesh-registration-and-property-prediction/models/cerulean-meadow-42'
liver_exp_name = 'classification_organ_liver_enc_channels_512_best_testacc_0.74.pth'
model_liver, test_loader_liver = return_data_model(liver_exp_dir,liver_exp_name, 'liver' )



AttributeError: 'Dict2Class' object has no attribute 'activation'

In [10]:
# SPLINE Model
spline_exp_dir = '/u/home/koksal/organ-mesh-registration-and-property-prediction/models/bright-universe-47'
spline_exp_name = 'classification_organ_spleen_enc_channels_512_best_testacc_0.70.pth'
model_spleen, test_loader_spleen = return_data_model(spline_exp_dir, spline_exp_name, 'spleen' )



In [ ]:
#Pancreas Model
pancreas_exp_dir = "/u/home/koksal/organ-mesh-registration-and-property-prediction/models/feasible-bush-46"
pancreas_exp_name = "classification_organ_pancreas_enc_channels_512_best_testacc_0.78.pth"
model_pancreas, test_loader_pancreas = return_data_model(pancreas_exp_dir, pancreas_exp_name, 'pancreas')

In [ ]:
#Left Kidney Mdeol
left_kidney_exp_dir = "/u/home/koksal/organ-mesh-registration-and-property-prediction/models/vocal-dragon-44"
left_kidney_exp_name = "classification_organ_left_kidney_enc_channels_512_best_testacc_0.75.pth"

model_left_kidney, test_loader_left_kidney = return_data_model(left_kidney_exp_dir, left_kidney_exp_name, 'left_kidney')

In [ ]:
# Right Kidney Model
right_kidney_exp_dir = '/u/home/koksal/organ-mesh-registration-and-property-prediction/models/ethereal-music-45'
right_kidney_exp_name = "classification_organ_right_kidney_enc_channels_512_best_testacc_0.69.pth"

model_right_kidney, test_loader_right_kidney = return_data_model(right_kidney_exp_dir, right_kidney_exp_name, 'right_kidney')

In [ ]:
model_pancreas.to(device).eval()
model_right_kidney.to(device).eval()
model_left_kidney.to(device).eval()
model_liver.to(device).eval()
model_spleen.to(device).eval()


In [ ]:
len(test_loader_pancreas)

In [ ]:
# Majority Voting
predictions_all= []
for i in range(len(test_loader_pancreas)):
    with torch.no_grad():

        sample_prediction = []
        #Pancreas Prediction
        batch_pancreas = test_loader_pancreas[i]
        prediction_pancreas = model_pancreas(batch_pancreas.to(device))
        gt_label_pancreas = torch.tensor(batch_pancreas.y,dtype=torch.int8).unsqueeze(0).to(device)
        predicted_seg_labels = torch.nn.Sigmoid()(prediction_pancreas)
        #predicted_seg_labels[predicted_seg_labels>0.5] = 1
        #predicted_seg_labels[predicted_seg_labels<0.5] = 0
        predicted_seg_label_pancreas = torch.round(predicted_seg_labels)

        sample_prediction.append(predicted_seg_label_pancreas)


        #Left Kidney Prediction
        batch_left_kidney = test_loader_left_kidney[i]
        prediction_left_kidney = model_left_kidney(batch_left_kidney.to(device))
        gt_label_left_kidney = torch.tensor(batch_left_kidney.y, dtype=torch.int8).unsqueeze(0).to(device)
        predicted_seg_labels = torch.nn.Sigmoid()(prediction_left_kidney)
        predicted_seg_label_left_kidney = torch.round(predicted_seg_labels)

        sample_prediction.append(predicted_seg_label_left_kidney)

        #Right Kidney Prediction
        batch_right_kidney = test_loader_right_kidney[i]
        prediction_right_kidney = model_right_kidney(batch_right_kidney.to(device))
        gt_label_right_kidney = torch.tensor(batch_right_kidney.y, dtype=torch.int8).unsqueeze(0).to(device)
        predicted_seg_labels = torch.nn.Sigmoid()(prediction_right_kidney)
        predicted_seg_label_right_kidney = torch.round(predicted_seg_labels)

        sample_prediction.append(predicted_seg_label_right_kidney)

        #Liver Prediction
        batch_liver = test_loader_liver[i]
        prediction_liver = model_liver(batch_liver.to(device)).to(device)
        gt_label_liver = torch.tensor(batch_liver.y, dtype=torch.int8).unsqueeze(0).to(device)
        predicted_seg_labels = torch.nn.Sigmoid()(prediction_liver)
        predicted_seg_label_liver = torch.round(predicted_seg_labels)

        sample_prediction.append(predicted_seg_label_liver)

        #Spleen Prediction
        batch_spleen = test_loader_spleen[i]
        prediction_spleen = model_spleen(batch_spleen.to(device)).to(device)
        gt_label_spleen = torch.tensor(batch_spleen.y, dtype=torch.int8).unsqueeze(0).to(device)
        predicted_seg_labels = torch.nn.Sigmoid()(prediction_spleen)
        predicted_seg_label_spleen = torch.round(predicted_seg_labels)

        sample_prediction.append(predicted_seg_label_spleen)




    predictions_all.append(sample_prediction)






In [ ]:
majority_predictions = []
# Majority Voting 
for sample_prediction in predictions_all:
    # Find most occuring label
    pred = torch.mode(torch.cat(sample_prediction).squeeze(1)).values.item()
    majority_predictions.append(pred)


In [ ]:
correct = 0
for i,item in enumerate(test_loader_liver):
    gt_label = item.y

    if gt_label == majority_predictions[i]:
        correct +=1 


    

In [ ]:
#Accuracy
correct/len(test_loader_pancreas)